In [1]:
import CodaPCA
import CodaRegress
import numpy as np
from runpca import read_csv
import os
import sklearn
import torch
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
#change module for newer sklearn versions
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection  import KFold
import matplotlib.pyplot as plt
import pandas as pd

W0705 16:28:28.496432 140087811557184 deprecation_wrapper.py:119] From /home/sean/Documents/Honours/honours/Code/coda-pca-orig/coda/codes/CodaPCA.py:14: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0705 16:28:28.497196 140087811557184 deprecation_wrapper.py:119] From /home/sean/Documents/Honours/honours/Code/coda-pca-orig/coda/codes/CodaPCA.py:14: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.



In [2]:
#read in the data. Given an array of which files are regression, classification or unlabelled
data_path = os.getcwd() + "/Aitchinson"

regression_list = [3,4,5,18,21,34,39]
classification_list = [7,8,9,11,12,16,17,19,23,24,25,26,28,29,33,37]
unlabelled_list=[1,2,6,10,13,14,15,20,22,27,30,31,32,35,36,38,40]

r_files = []
c_files = []
u_files = []


for file in os.listdir(data_path):
    for i in regression_list:
        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:
            r_files.append("Aitchinson/" + file)
    for i in classification_list:
        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:
            c_files.append("Aitchinson/" + file)
    for i in unlabelled_list:
        if os.path.isfile(os.path.join(data_path,file)) and 'Data ' + str(i) + '.' in file:
            u_files.append("Aitchinson/" + file)

In [3]:
def coda_val(features, targets, n_components, folds):
    targets = targets.reshape(-1,1)
    param_splits = 3
    kfold_scores = []
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
       
        Y_test = targets[test]
        X_test = features[test]
        
        
        kf_inner = KFold(param_splits)
        
        inner_folds = [i for i in kf_inner.split(X_train)]      
            
        #inner loop for parameter selection (lambda term in combined loss):
        param_grid = [0.01,0.05,0.1,0.5,1.0,5.0,10.0,50.0,100.0]
        
        
        for a in param_grid:
            max_error = np.inf
            
   
            
            cval_error = []
            #find the parameter which obtains the best inner cross val score
            for train_inner, test_inner in inner_folds:
                
                model = CodaRegress.CoDA_Regress(features.shape[1], n_components, [10,], [2,])
                
                model.fit(torch.FloatTensor(X_train[train_inner]),  torch.FloatTensor(Y_train[train_inner]), a, lr=1e-2)

                y_inner_pred = model.predict(torch.FloatTensor(X_train[test_inner]))
    
                cval_error.append(sklearn.metrics.mean_squared_error(Y_train[test_inner],y_inner_pred.detach().numpy()))
        
        
                
    
   
            curr_error = np.mean(cval_error)
            if curr_error < max_error:
                max_error = curr_error
                best_param = a
                print ("Current best", best_param)
        
        #compute test score based on best parameter
        best_model = CodaRegress.CoDA_Regress(features.shape[1], n_components, [10,], [2,])
        best_model.fit(torch.FloatTensor(X_train),  torch.FloatTensor(Y_train), a, lr=1e-2)

        y_pred = best_model.predict(torch.FloatTensor(X_test))
        kfold_scores.append(sklearn.metrics.mean_squared_error(Y_test,y_pred.detach().numpy()))
                
    return kfold_scores

    
    
    

In [4]:

#need to specify where the the targets and features are in the dataset, and whether there are non compositional features

def PCA_Regression(data, co_feature_indices, target_index, 
                   other_feature_indices = [], alg=CodaPCA.Alg.CODAPCA, verbose=True):
    
    #can loop through/optimise this in another way?
    
    headers = data[1]
    features = data[0][:,co_feature_indices]
    targets = data[0][:,target_index]
    
    #normalise the compositional features. TODO anything extra to deal with non compositional features?
    features = np.array([feat/sum(feat) for feat in features])

    #can be empty
    extra_features = data[0][:,other_feature_indices]
    
    #TODO double check this
    features = np.hstack([features, extra_features])
    
    #compute the CoDA-PCA projection 
    #TODO add component number as a hyperparameter to optimise 
    n_components=len(co_feature_indices)-2

    pca = CodaPCA.CodaPCA(n_components,lrate=1e-4,nn_shape=[100,100], alg=alg)
    #TODO: check why this is numerically unstable
    #pca = CodaPCA.NonParametricCodaPCA(n_components)

    pca.fit(features)
    
    Y_coda = pca.transform(features)

    pca_clr = CodaPCA.CLRPCA(n_components)
    pca_clr.fit(features)
    
    Y_clr = pca_clr.transform(features)
    
    
    splits = 4
        
    #split data 
    kf = KFold(splits)
        
    folds = [i for i in kf.split(features)]      
    

    lm = Ridge()
    
    coda_score = enhanced_cross_val(lm,Y_coda, targets, folds)
    clr_score = enhanced_cross_val(lm,Y_clr, targets, folds) 
    naive_score = enhanced_cross_val(lm, features, targets, folds)
    
    regress_score = coda_val(features, targets, n_components, folds)
    


    if verbose:
        print("CoDA-PCA:")
        print(coda_score)
        print("CLR-PCA:")
        print(clr_score)
        print ("Naive regression:")
        print (naive_score)
        print ("CoDA-Regress:")
        print (regress_score)
    

    return coda_score,clr_score,naive_score,regress_score

#training methodology as described in:
#https://papers.nips.cc/paper/3215-learning-with-transformation-invariant-kernels.pdf
def enhanced_cross_val(model, features, targets, folds):
    assert len(features) == len(targets), "Mismatch in length of features and targets"
    
    #define the number of splits and folds uised in the parameter selection
    #stick to smaller splits since we have small datasets
    #splits = 4
    param_splits = 3
    #split data 
    #kf = KFold(splits)
    kfold_scores = []
    for train, test in folds:        
        Y_train = targets[train]
        X_train = features[train]
        
       
        Y_test = targets[test]
        X_test = features[test]
        
        #inner loop for parameter selection (regularisation term in Ridge Regression):
        param_grid = [0.01,0.05,0.1,0.5,1.0,5.0,10.0,50.0,100.0]
        for a in param_grid:
            max_score = -np.inf
            lm = Ridge(a)
   
            curr_score = np.mean(cross_val_score(lm, X_train, Y_train,cv=param_splits))
            if curr_score > max_score:
                max_score = curr_score
                best_param = a
        
        #compute test score based on best parameter
        lm = Ridge(best_param)
        lm.fit(X_train, Y_train)
        y_pred = lm.predict(X_test)
        kfold_scores.append(sklearn.metrics.mean_squared_error(Y_test,y_pred))
                
    return kfold_scores



#can automate this if we had assume a certain structure for the indices of features and targets, or an array per dataset 

score_dict = {}

#TODO r_files isn't always consistent

data_21_scores = PCA_Regression(read_csv(r_files[0], normalize=False), co_feature_indices=[0,1,2,3], target_index=4)
score_dict['21'] = data_21_scores

data_21_scores2 = PCA_Regression(read_csv(r_files[0], normalize=False), co_feature_indices=[0,1,2,3], target_index=5)
score_dict['21,2'] = data_21_scores2

data_34_scores = PCA_Regression(read_csv(r_files[1], normalize=False), co_feature_indices=[0,1,2,3], target_index=4) 
score_dict['34'] = data_34_scores

data_3_scores = PCA_Regression(read_csv(r_files[2], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5) 
score_dict['3'] = data_3_scores

data_39_scores = PCA_Regression(read_csv(r_files[3], normalize=False), co_feature_indices=[0,1,2], target_index=3)
score_dict['39'] = data_39_scores

data_4_scores = PCA_Regression(read_csv(r_files[4], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=5)
score_dict['4'] = data_4_scores

data_4_scores2 = PCA_Regression(read_csv(r_files[4], normalize=False), co_feature_indices=[0,1,2,3,4], target_index=6)
score_dict['4,2'] = data_4_scores2

data_5_scores = PCA_Regression(read_csv(r_files[5], normalize=False), co_feature_indices=[0,1,2], target_index=3)
score_dict['5'] = data_4_scores

data_18_scores = PCA_Regression(read_csv(r_files[6], normalize=False), co_feature_indices=[0,1,2], target_index=3)
score_dict['18'] = data_5_scores


loading Aitchinson/Data 21. Permeabilities of bayesite for 21 mixtures of fibres and bonding pressures..csv...
21 samples 6 features
sparsity: 0.0%
[epoch     0] L=  3.9608
epoch 10000, loss 1.352860927581787
epoch 10000, loss 0.37952518463134766
epoch 10000, loss 1.2401155233383179
Current best 0.01
epoch 10000, loss 1.9284913539886475
epoch 10000, loss 1.8880974054336548
epoch 10000, loss 2.592592716217041
Current best 0.05
epoch 10000, loss 4.752969741821289
epoch 10000, loss 3.771939754486084
epoch 10000, loss 4.553487300872803
Current best 0.1
epoch 10000, loss 19.803091049194336
epoch 10000, loss 18.811616897583008
epoch 10000, loss 18.56463623046875
Current best 0.5
epoch 10000, loss 38.04826736450195
epoch 10000, loss 37.60554885864258
epoch 10000, loss 37.873409271240234
Current best 1.0
epoch 10000, loss 186.2340087890625
epoch 10000, loss 186.15283203125
epoch 10000, loss 183.7068634033203
Current best 5.0
epoch 10000, loss 370.5549011230469
epoch 10000, loss 369.43719482421

epoch 10000, loss 200.0272216796875
Current best 5.0
epoch 10000, loss 6913.39599609375
epoch 10000, loss 1153.532958984375
epoch 10000, loss 8619.7841796875
Current best 10.0
epoch 10000, loss 2287.504638671875
epoch 10000, loss 2715.96875
epoch 10000, loss 3067.94189453125
Current best 50.0
epoch 10000, loss 3825.27685546875
epoch 10000, loss 4667.95068359375
epoch 10000, loss 4949.87109375
Current best 100.0
epoch 10000, loss 7283.83447265625
epoch 10000, loss 648.259521484375
epoch 10000, loss 8538.2412109375
epoch 10000, loss 0.39940282702445984
Current best 0.01
epoch 10000, loss 649.7942504882812
epoch 10000, loss 456.390625
epoch 10000, loss 785.1847534179688
Current best 0.05
epoch 10000, loss 651.7039794921875
epoch 10000, loss 8550.830078125
epoch 10000, loss 8203.2138671875
Current best 0.1
epoch 10000, loss 666.9908447265625
epoch 10000, loss 474.6551513671875
epoch 10000, loss 20.135704040527344
Current best 0.5
epoch 10000, loss 686.088134765625
epoch 10000, loss 492.335

epoch 10000, loss 24.922040939331055
epoch 10000, loss 25.27458953857422
Current best 0.5
epoch 10000, loss 49.27713394165039
epoch 10000, loss 60.38109588623047
epoch 10000, loss 61.37883758544922
Current best 1.0
epoch 10000, loss 232.16317749023438
epoch 10000, loss 248.79904174804688
epoch 10000, loss 248.7668914794922
Current best 5.0
epoch 10000, loss 457.16754150390625
epoch 10000, loss 498.136474609375
epoch 10000, loss 495.71728515625
Current best 10.0
epoch 10000, loss 2276.17919921875
epoch 10000, loss 2487.893798828125
epoch 10000, loss 2476.24462890625
Current best 50.0
epoch 10000, loss 4546.40576171875
epoch 10000, loss 4975.26953125
epoch 10000, loss 4952.052734375
Current best 100.0
epoch 10000, loss 7252.83349609375
epoch 10000, loss 0.46492159366607666
epoch 10000, loss 0.49765533208847046
epoch 10000, loss 0.5101235508918762
Current best 0.01
epoch 10000, loss 9.314949989318848
epoch 10000, loss 2.5132832527160645
epoch 10000, loss 2.5169312953948975
Current best 0.

[epoch     0] L=  4.2066
epoch 10000, loss 2.852203607559204
epoch 10000, loss 3.242548704147339
epoch 10000, loss 0.9063756465911865
Current best 0.01
epoch 10000, loss 13.60297966003418
epoch 10000, loss 13.733473777770996
epoch 10000, loss 2.5717506408691406
Current best 0.05
epoch 10000, loss 5.784623146057129
epoch 10000, loss 5.241025924682617
epoch 10000, loss 3.6351211071014404
Current best 0.1
epoch 10000, loss 19.249937057495117
epoch 10000, loss 18.983810424804688
epoch 10000, loss 20.87687110900879
Current best 0.5
epoch 10000, loss 42.73836135864258
epoch 10000, loss 36.243873596191406
epoch 10000, loss 35.46378707885742
Current best 1.0
epoch 10000, loss 172.97796630859375
epoch 10000, loss 170.5129852294922
epoch 10000, loss 174.11720275878906
Current best 5.0
epoch 10000, loss 335.3902587890625
epoch 10000, loss 354.0248107910156
epoch 10000, loss 351.64263916015625
Current best 10.0
epoch 10000, loss 1683.796875
epoch 10000, loss 1703.63916015625
epoch 10000, loss 1743

epoch 10000, loss 340.0734558105469
epoch 10000, loss 354.24176025390625
epoch 10000, loss 367.57366943359375
Current best 10.0
epoch 10000, loss 1699.6719970703125
epoch 10000, loss 1769.66943359375
epoch 10000, loss 1836.65869140625
Current best 50.0
epoch 10000, loss 3399.187255859375
epoch 10000, loss 3538.567138671875
epoch 10000, loss 3672.991455078125
Current best 100.0
epoch 10000, loss 5305.3408203125
epoch 10000, loss 0.5915667414665222
epoch 10000, loss 0.6770172119140625
epoch 10000, loss 0.6673576831817627
Current best 0.01
epoch 10000, loss 1.9878535270690918
epoch 10000, loss 2.133329153060913
epoch 10000, loss 2.13737154006958
Current best 0.05
epoch 10000, loss 3.7181525230407715
epoch 10000, loss 3.948502540588379
epoch 10000, loss 3.975196123123169
Current best 0.1
epoch 10000, loss 17.656967163085938
epoch 10000, loss 18.481124877929688
epoch 10000, loss 18.623790740966797
Current best 0.5
epoch 10000, loss 35.071346282958984
epoch 10000, loss 36.518314361572266
epo

epoch 10000, loss -504853664.0
epoch 10000, loss -422340832.0
epoch 10000, loss -753893504.0
Current best 0.05
epoch 10000, loss -1013793600.0
epoch 10000, loss -842072640.0
epoch 10000, loss -1522960768.0
Current best 0.1
epoch 10000, loss -5054588416.0
epoch 10000, loss -4229857792.0
epoch 10000, loss -7603624448.0
Current best 0.5
epoch 10000, loss -10137572352.0
epoch 10000, loss -8415312896.0
epoch 10000, loss -15087070208.0
Current best 1.0
epoch 10000, loss -50821820416.0
epoch 10000, loss -42240327680.0
epoch 10000, loss -76136792064.0
Current best 5.0
epoch 10000, loss -101031870464.0
epoch 10000, loss -84124680192.0
epoch 10000, loss -152087756800.0
Current best 10.0
epoch 10000, loss -505607225344.0
epoch 10000, loss -420497948672.0
epoch 10000, loss -761370836992.0
Current best 50.0
epoch 10000, loss -1011108151296.0
epoch 10000, loss -840127086592.0
epoch 10000, loss -1513601892352.0
Current best 100.0
epoch 10000, loss -1681920098304.0
epoch 10000, loss -50641388.0
epoch 

In [1]:
#note: plotly code works fine, but gives a jupyter warning when saving with a rendered table

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


init_notebook_mode(connected=True)


table_rows = []
for key in score_dict.keys():
    mean_scores = list(map(np.mean, score_dict[key]))
    table_rows.append([key,*mean_scores])
    
results = [go.Table(
    header=dict(values=["Dataset","CoDA-PCA", "CLR-PCA", "Naive Regression", "CoDA-Regress"]),
    cells=dict(values=np.array(table_rows).T))]

iplot(results, filename = 'basic_table')    
    


NameError: name 'score_dict' is not defined